Plan:
* Use linspace to produce a bunch of values at some intervals
* SMARTLY restrict values
* Try and make it make sense
* Code equations first!
* Could brute force with REAL values (restrict to range) or CALCULATE values (use equations from geometry)

## Equations
With credit to tables from _Design of Rockets and Space Launch Vehicles_
* Change in velocity: $$\Delta v = v_{eq}ln(\frac{m_0}{m_f}) = v_{eq}ln(mass\ ratio) = v_{eq}ln(\mu) = g_0 I_{sp}ln\mu$$
    * $g_0$ is Earth's standard gravity at sea level. Taken to be 9.80665 $m/s^2$
* Missing time to burn equation
* Mass fraction: $$\mu = \frac{initial\ mass}{final\ mass} = \frac{m_0}{m_f} = \frac{m_0}{m_0-m_p}$$ 
<br />
* Total Launch Weight (Propellant and shell - NOT payload):
$$W_L = 0.04 Ld^2$$ 
    * NOTE: lb, in, in
    * Can be used to calculate masses instead of using values
<br /><br />
* Body Structure vs. Launch Weight: $$W_{Body Structure}/W_{Launch Weight} = 0.22$$
    * Launch weight includes the propellant
    * Combine with Total launch weight equations

### SM3 Mass Fractions (Mf/Mo)
* First Stage: .376
* Second Stage: .612
* Third Stage: .604

### Range of Real Mass Fractions (Mf/Mo)
With credit to tables from _Design of Rockets and Space Launch Vehicles_
* First Stage: 
    * 0.0432 -> 0.1198
* Second Stage:
    * 0.0495 -> 0.1700
* Third Stage:
    * 0.0824 -> 0.1008

### Range of Isps
With credit to tables from _Design of Rockets and Space Launch Vehicles_
* 200 seconds to 250 seconds 
    * 250 seconds more applicable

In [2]:
import numpy as np


In [3]:
specific_impulse = 250 # seconds
g = 9.80665 # m / s^2
v_eq = g*specific_impulse

meters_to_inches = 39.3700787402 ## multiply to meters
kg_to_lbs = 2.20462

payload = 250 * kg_to_lbs  ## in lbs


## Calculating Delta V using range of actual values

In [4]:
#m0 / mf
mass_frac_first = 1/np.linspace(0.0400, 0.1200)
mass_frac_second = 1/np.linspace(.0450, 0.1700) 
mass_frac_third = 1/np.linspace(.0800, 0.1010)

In [5]:
max_mf1 = np.max(mass_frac_first)
max_mf2 = np.max(mass_frac_second)
max_mf3 = np.max(mass_frac_third)

In [6]:
deltav_first = np.log(mass_frac_first) * v_eq
deltav_second = np.log(mass_frac_second) * v_eq
deltav_third = np.log(mass_frac_third) * v_eq
deltav_total_max = np.max(deltav_first)+np.max(deltav_second)+np.max(deltav_third)

In [7]:
print(
"Unitless values.\nMax Mass Fraction\n"
    "\tFirst: ", max_mf1, "\n\tSecond: ", max_mf2, "\n\tThird: ", max_mf3)
print(
"\n\nUnits in meters per second.\nMax Delta V\n",
    "\tFirst: ", np.max(deltav_first), "\n\tSecond: ", np.max(deltav_second), "\n\tThird: ", np.max(deltav_third),
"\n\nTotal Delta V: ", deltav_total_max)

Unitless values.
Max Mass Fraction
	First:  25.0 
	Second:  22.22222222222222 
	Third:  12.5


Units in meters per second.
Max Delta V
 	First:  7891.597151985934 
	Second:  7602.832900331016 
	Third:  6192.234202426388 

Total Delta V:  21686.66425474334


## Calculating Delta V with Mass Equations

In [8]:
## Constants given in project

total_length = 10.0 * 39.3700787402
diameter = 1.0 * 39.3700787402 ## convert given meters to inches (final unit inches)

In [9]:
launch_weight = 0.04 * total_length * (diameter)*(diameter) ## pounds

In [10]:
print(launch_weight, "lbs")
print(launch_weight/2.205, "kg")

24409.497637972003 lbs
11070.066955996374 kg


In [11]:
## Lengths in ratio form
l1_ratio = np.linspace(0.1, 1, 100)
l2_ratio = np.linspace(0.1, 1, 100)
#print(np.round(l1_ratio*10, 1)) ## lengths corresponding to l1's ratio, rounded to tenths

In [12]:
## Create new data type to hold a tuple in each element of the array 
##      (makes adding elements easier)

## length 1 ratio, length 2 ratio, length 3 ratio
dt = np.dtype([('l1r', np.float64), ('l2r', np.float64), ('l3r', np.float64)])

## Test dt to make sure it works as expected

test = np.array([(1,2,3), (1,3,4)], dtype = dt)

print(test)
np.append(test, np.array([(4.,4.,2.)], dtype = dt))


[(1., 2., 3.) (1., 3., 4.)]


array([(1., 2., 3.), (1., 3., 4.), (4., 4., 2.)],
      dtype=[('l1r', '<f8'), ('l2r', '<f8'), ('l3r', '<f8')])

In [13]:
## Find all possible ratio combos between the three stages, put it in an np array of tuples
ratio_list = np.array([], dtype=dt )

for i in l1_ratio:
    for j in l2_ratio:
        if(i+j > 1):
            continue
        else:
            ratio_list = np.append(ratio_list, np.array([(i, j, 1-i-j)], dtype = dt)) 
    
    
    

In [14]:
## convert the np array of tuples to a more useful np array of 3 columns and each row representing a tuple of 
## the ratios
ratio_list_converted = np.array([])
for i in ratio_list:
    ratio_list_converted = np.append(ratio_list_converted, np.array([[i[0], i[1], i[2]]]))
    
ratio_list_converted = np.reshape(ratio_list_converted, (-1, 3)).astype(np.float64)

##np.set_printoptions(threshold=np.inf) - to see ENTIRE list
print(np.round(ratio_list_converted*10, 2), "\n")
print(ratio_list_converted.shape)

## 4005 different combinations!

[[1.   1.   8.  ]
 [1.   1.09 7.91]
 [1.   1.18 7.82]
 ...
 [8.91 1.   0.09]
 [8.91 1.09 0.  ]
 [9.   1.   0.  ]] 

(4005, 3)


In [16]:
##now we calculate masses

launch_weights = 0.04 * (10 * ratio_list_converted * meters_to_inches) * diameter * diameter + payload ## lbs 

final_masses = launch_weights*.22 + payload ## after losing the propellant

In [17]:
test1 = np.array([[0, 2], [4,5]])
print(np.sum(test1, axis = 1))

[2 9]


In [18]:
## now mass fractions - m0/mf

stage_1_mf = np.sum(launch_weights, axis = 1)/(np.sum(launch_weights[:, 1:2], axis=1) + final_masses[:, 0])

stage_2_mf = np.sum(launch_weights[:, 1:2], axis = 1)/(launch_weights[:, 2] + final_masses[:, 1])

stage_3_mf = launch_weights[:, 2]/final_masses[:, 2]

## Calculate mass fractions by taking initial launch weight / final masses of the lowest stage

mass_fractions = np.stack((stage_1_mf, stage_2_mf, stage_3_mf), axis = 1)

print(mass_fractions.shape)

print(mass_fractions[:, 0])
print(stage_1_mf)

print(mass_fractions[:, 1])
print(stage_2_mf)

print(mass_fractions[:, 2])
print(stage_3_mf)



(4005, 3)
[6.20321817 5.89202911 5.61057066 ... 3.08482134 3.005873   3.06709893]
[6.20321817 5.89202911 5.61057066 ... 3.08482134 3.005873   3.06709893]
[0.14055246 0.15221389 0.16406808 ... 1.50927547 1.77629237 1.69950617]
[0.14055246 0.15221389 0.16406808 ... 1.50927547 1.77629237 1.69950617]
[4.04122596 4.03622237 4.03111849 ... 1.07186552 0.81967213 0.81967213]
[4.04122596 4.03622237 4.03111849 ... 1.07186552 0.81967213 0.81967213]


In [19]:
## now delta vs

delta_v = np.log(mass_fractions) * v_eq 
delta_v.shape
## and quickly sum for total delta vs

delta_v = np.sum(delta_v, axis = 1)



In [20]:
## Now find best performing mass fractions

print(delta_v)
print(np.max(delta_v))
print(np.argmax(delta_v))

best_performing_index = np.argmax(delta_v)

print(mass_fractions[best_performing_index, :])


[3087.7262756  3153.91928492 3214.67507849 ... 3941.10645056 3619.25557931
 3560.35087179]
5209.8135273267835
2912
[1.85917294 1.82037589 2.47400373]


In [21]:
## What if I want to find the next best performing? The next 5 performing?

sorted_delta_v = np.sort(delta_v)

##reverse it to get descending instead of ascending

sorted_delta_v = sorted_delta_v[::-1]

## now it stores biggest first and next biggest after

print(sorted_delta_v[0:10])

for i in sorted_delta_v[0:100]:
    print(i)
    print(mass_fractions[np.where(delta_v == i)[0], :])



[5209.81352733 5209.61288943 5209.51203678 5208.94007612 5208.67620936
 5207.82297567 5207.59932598 5207.46481957 5207.27136907 5207.21045609]
5209.8135273267835
[[1.85917294 1.82037589 2.47400373]]
5209.612889425562
[[1.83649794 1.84270102 2.47400373]]
5209.512036779421
[[1.88241487 1.79767887 2.47400373]]
5208.940076122227
[[1.81436938 1.86466333 2.47400373]]
5208.676209361527
[[1.90624525 1.77460059 2.47400373]]
5207.822975674914
[[1.79276775 1.88627159 2.47400373]]
5207.599325982592
[[1.87580082 1.86841486 2.38687102]]
5207.464819570723
[[1.89946299 1.84503824 2.38687102]]
5207.271369074668
[[1.93068674 1.75113136 2.47400373]]
5207.210456091979
[[1.85272093 1.89139023 2.38687102]]
5206.802819746003
[[1.84283726 1.77654513 2.55437503]]
5206.772511445591
[[1.92372975 1.82124976 2.38687102]]
5206.743372429091
[[1.8205566  1.79824356 2.55437503]]
5206.381010368192
[[1.86567004 1.75450121 2.55437503]]
5206.330174527707
[[1.83020209 1.91397458 2.38687102]]
5206.287583074582
[[1.77167443 